# Revocation - Alice

## Role: Issuer & Verifier

In this notebook you will issue a revocable credential to [Bob](http://127.0.0.1:8889/notebooks/Part%2010%20-%20Revocation.ipynb). Bob will store this credential and present proof of it back to Alice, where she will verify the presentation. Alice will then revoke the credential and request another presentation of the credential from Bob. This presentation will not verify.

## How Revocation Works

Revocation is quite complex in the Hyperledger stack. This is because to protect the privacy of credential holders they have chosen a cryptographic solution that allows issuers to publish revocations such that holders can prove non-revocation without a verifier needing to communicate with the issuer. 

An easy way to achieve this would be to include an index for each credential, then publish the list of revoked credential indexes. However, such an index is equivalent to a correlatable identifiers for that credential. 

Tails File. ...


## 1. Initiate the controller for Alice

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)

IPython autoawait is `on`, and set to use `asyncio`


## 2. Register Listeners

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}

agent_controller.register_listeners([cred_listener, proof_listener], defaults=True)

## 3. Check the agent has an active connection

**Note: An active connection is required, this should have been established on start up through the python script create_connection.py in the setup folder. If not it is possible to run through the did-exchange tutorial to create one between Alice and Bob**

* [Alice](http://127.0.0.1:8888/notebooks/Part%203%20-%20Establishing%20a%20Connection.ipynb)
* [Bob](http://127.0.0.1:8889/notebooks/Part%203%20-%20Establishing%20a%20Connection.ipynb)

In [3]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    

Results :  [{'my_did': '26e44fuEpzT4UPQLqpsckm', 'invitation_key': 'CVC3TbN5soQdwq5idwmWxqFJadSMz5KeN3aXujW9QVR6', 'invitation_mode': 'once', 'state': 'active', 'their_label': 'Bob', 'request_id': 'f78440d6-4f07-42e6-b6a2-dbd24e3ed7f4', 'their_did': 'FCMbQ4KnMWo1AEXTXZnUt9', 'routing_state': 'none', 'accept': 'manual', 'initiator': 'external', 'updated_at': '2020-12-09 17:40:52.619502Z', 'connection_id': 'dc7a71b8-41d3-4720-b36e-cd6856f61bc3', 'created_at': '2020-12-09 17:40:32.408030Z'}]
Connection : {'my_did': '26e44fuEpzT4UPQLqpsckm', 'invitation_key': 'CVC3TbN5soQdwq5idwmWxqFJadSMz5KeN3aXujW9QVR6', 'invitation_mode': 'once', 'state': 'active', 'their_label': 'Bob', 'request_id': 'f78440d6-4f07-42e6-b6a2-dbd24e3ed7f4', 'their_did': 'FCMbQ4KnMWo1AEXTXZnUt9', 'routing_state': 'none', 'accept': 'manual', 'initiator': 'external', 'updated_at': '2020-12-09 17:40:52.619502Z', 'connection_id': 'dc7a71b8-41d3-4720-b36e-cd6856f61bc3', 'created_at': '2020-12-09 17:40:32.408030Z'}
Active Conne

## 4. Write a Schema to the Ledger

**Note: You will only be able to do this once unless you change the schema_name or version. Or tear down the current ledger using ./manage down**

In [4]:
# Define you schema name - must be unique on the ledger
schema_name = "test_revocable_schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "skill", "age"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)


PQRXDxdGqQGSZ8z69p4xZP:2:test_revocable_schema:0.0.1


## 5. Write a Revocable Credential Definition to the Ledger

By adding the support_revocation flag, the ACA-Py agent under the hood knows to create some additional structures to support revocation of all credentials issued under this cred_def.

Specifically the agent creates what is known as a revocation registry, and writes this to the public ledger. You should be able to see these transactions in the ledger explorer at [localhost:9000](http://localhost:9000/browse/domain).

It also add's a tails file to the tail server that needs to be accessible by those creating and verifying proofs of non-revocation.

In [5]:
response = await agent_controller.definitions.write_cred_def(schema_id, support_revocation=True)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default


## 6. Populate the Attribues to Issue to Bob

In [6]:
credential_attributes = [
    {"name": "name", "value": "Bob"},
    {"name": "skill", "value": "revocation"},
    {"name": "age", "value": "1337"}
]
print(credential_attributes)

[{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'revocation'}, {'name': 'age', 'value': '1337'}]


## 7. Continue in [Bob's Notebook](http://127.0.0.1:8889/notebooks/Part%2010%20-%20Revocation.ipynb)

You need to initialise the controller and listen to webhooks so you can track the messages Bob's agent receives.

## 9. Issue Revocable Credential

Note: It is exactly the same as issuing a unrevocable credential. Revocable credentials are those issued under cred_defs (public keys) that have been appropriately set up to support the revocation cryptography.

In [7]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 6b4993c9-a553-4672-915d-df9d07b5dffa, role: issuer, state: offer_sent
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'revocation'}, {'name': 'age', 'value': '1337'}]
Credential exchange 6b4993c9-a553-4672-915d-df9d07b5dffa, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 6b4993c9-a553-4672-915d-df9d07b5dffa, role: issuer, state: request_received
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'revocation'}, {'name': 'age', 'value': '1337'}]
Handle Credentials
Credential exchange 6b4993c9-a553-4672-915d-df9d07b5dffa, role: issuer, state: credential_issued
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'revocation'}, {'name': 'age', 'value': '1337'}]
Handle Credentials
Credential exchange 6b4993c9-a553-4672-915d-df9d07b5dffa, role: issuer, state: credential_acked
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'revocation'}, {'nam

## Continue in [Bob's Notebook ](http://127.0.0.1:8889/notebooks/Part%2010%20-%20Revocation.ipynb)

Here you will request, store and then present the credential.

## 14. Send Request in referencing received proposal

In [8]:
response = await agent_controller.wallet.get_public_did()
print(response)
issuer_did = response["result"]["did"]

print("Request proof of Skill from Bob")
#Set some variables

SELF_ATTESTED = True
exchange_tracing = False

# Either the attribute can be specified to be non revoked
req_attrs = [
    {
        "name": "skill",
        "restrictions": [{"issuer_did": issuer_did}],
        "non_revoked": {"to": int(time.time() - 1)},
    }
]



indy_proof_request = {
    "name": "Proof of Personal Information",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
    },
}


## Or the overall request itself
indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

response = await agent_controller.proofs.send_request(proof_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)

{'result': {'did': 'PQRXDxdGqQGSZ8z69p4xZP', 'verkey': 'DDEKJtBjzaXAJtpwetdPiXH5s4rNUEzG18V15QoLRcZZ', 'posture': 'public'}}
Request proof of Skill from Bob
{'state': 'request_sent', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': 'e16c2be3-7ae1-4df6-b25c-bbcbce108396', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIFBlcnNvbmFsIEluZm9ybWF0aW9uIiwgInZlcnNpb24iOiAiMS4wIiwgInJlcXVlc3RlZF9hdHRyaWJ1dGVzIjogeyIwX3NraWxsX3V1aWQiOiB7Im5hbWUiOiAic2tpbGwiLCAicmVzdHJpY3Rpb25zIjogW3siaXNzdWVyX2RpZCI6ICJQUVJYRHhkR3FRR1NaOHo2OXA0eFpQIn1dLCAibm9uX3Jldm9rZWQiOiB7InRvIjogMTYwNzUzNTk0NX19fSwgInJlcXVlc3RlZF9wcmVkaWNhdGVzIjoge30sICJub25fcmV2b2tlZCI6IHsidG8iOiAxNjA3NTM1OTQ2fSwgIm5vbmNlIjogIjQwMjk5NTkxNjY5OTM4NTE2NDEwODE1In0='}}]}, 'presentation_exchange_id': 'ef73a9b3-c8bf-4110-80d5-4eda25c37fed', 'auto_present': False, 'trace'

## Continue in Bob's notebook to respond to the presentation request

## 17. Verify the Presentation

In [9]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
print(verify['state'] == 'verified')
for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print(name + " : " + val['raw'])

Handle present proof
Role verifier, Exchange ef73a9b3-c8bf-4110-80d5-4eda25c37fed in state verified
True
0_skill_uuid : revocation


## 18. Revoke the Credential

In [16]:
active_rev_reg = await agent_controller.revocations.get_active_revocation_registry_by_cred_def(cred_def_id)
rev_reg_id = active_rev_reg["result"]["revoc_reg_def"]["id"]
rev_reg_id
active_rev_reg

{'result': {'max_cred_num': 1000,
  'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP',
  'tails_public_uri': 'https://a5e208692130.ngrok.io/PQRXDxdGqQGSZ8z69p4xZP:4:PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default:CL_ACCUM:60dd0fff-44f9-4971-802a-bf5114cc379b',
  'state': 'active',
  'revoc_reg_id': 'PQRXDxdGqQGSZ8z69p4xZP:4:PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default:CL_ACCUM:60dd0fff-44f9-4971-802a-bf5114cc379b',
  'revoc_def_type': 'CL_ACCUM',
  'pending_pub': [],
  'record_id': '60dd0fff-44f9-4971-802a-bf5114cc379b',
  'revoc_reg_entry': {'ver': '1.0',
   'value': {'accum': '21 110C86416DB2D469C83A52FD12496F95051B9F97085D955D712D5A00BECB4B481 21 13FD510CFEB72AD96DBD7E18894190224B09FABF5F839313920320B54C057FCCC 6 74839E0ED76F95F6EFDB4CB670D919F557D1EA51FB27049B6BAEF29DCF7F0D92 4 451E848F716BE55FAD77B2ADD693C90102358CEA41DA0B113593460F0B727D39 6 65E1E7ED9BA5B340A43B2006B976A18BD5D3E5E599A33CEE305E54F8EF24BAC1 4 14B5560549D93ED83DC996AF8DC57D25A7F99164A5B78AD3915094C9A9B829DF'}},
  'tails_hash': '8jZjLAjoZGqbN

In [13]:
response = await agent_controller.revocations.revoke_credential(record_id, "1", rev_reg_id, publish=True)

ClientResponseError: 422, message='Unprocessable Entity', url=URL('http://alice-agent:8021/revocation/revoke')

Error during POST /revocation/revoke: 422, message='Unprocessable Entity', url=URL('http://alice-agent:8021/revocation/revoke')
